In [61]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn import ensemble, preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


In [62]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [63]:
train.isnull().sum()

portfolio_id         0
desk_id           3665
office_id            0
pf_category          0
start_date           0
sold                 2
country_code         0
euribor_rate         0
currency             0
libor_rate         474
bought               2
creation_date        0
indicator_code    5699
sell_date            0
type                 0
hedge_value       5701
status            3084
return               0
dtype: int64

In [64]:
train.head(10)

,portfolio_id,desk_id,office_id,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status,return
0,PF00001002,DSK00001001,OFF00001002,B,20040720,110000000.0,T,0.02074,USD,2.332216,1.098097e+08,20040720,NaN,20040812,B,NaN,NaN,0.02496
1,PF00001003,DSK00001002,OFF00001001,A,20040709,176671000.0,N,0.02074,GBP,5.269617,1.760084e+08,20040723,NaN,20040812,C,NaN,NaN,0.05496
2,PF00001005,DSK00001004,OFF00001001,A,20040723,56474000.0,T,0.02074,USD,2.332216,5.637953e+07,20040723,NaN,20040817,A,NaN,NaN,0.02496
3,PF00001006,DSK00001005,OFF00001001,A,20040609,164813000.0,T,0.02074,USD,2.332216,1.645088e+08,20040723,NaN,20040713,A,NaN,NaN,0.02496
4,PF00001007,DSK00001005,OFF00001002,B,20040609,140800000.0,T,0.02074,USD,2.332216,1.405402e+08,20040723,NaN,20040713,B,NaN,NaN,0.02496
5,PF00001008,DSK00001006,OFF00001001,A,20040707,48741000.0,T,0.02074,USD,2.332216,4.865127e+07,20040726,NaN,20040810,A,NaN,NaN,0.02490
6,PF00001010,DSK00001009,OFF00001001,A,20040706,60593500.0,T,0.02074,USD,2.332216,6.048181e+07,20040726,NaN,20040809,A,NaN,NaN,0.02493
7,PF00001011,DSK00001009,OFF00001002,B,20040706,134200000.0,T,0.02074,USD,2.332216,1.339526e+08,20040726,NaN,20040809,B,NaN,NaN,0.02493
8,PF00001012,DSK00001010,OFF00001001,A,20040419,82929000.0,T,0.02074,USD,2.332216,8.250616e+07,20040726,NaN,20040720,A,NaN,NaN,0.02460
9,PF00001016,DSK00001014,OFF00001001,A,20040414,212476000.0,T,0.02074,USD,2.332216,2.114140e+08,20040727,NaN,20040713,A,NaN,NaN,0.02466


In [65]:
col_more = []
col_less = []
train.replace(np.nan,-1,inplace= True)
test.replace(np.nan,-1,inplace= True)
for i in train.columns:
    if train[i].dtype == 'object':
        if len(np.unique(list(train[i])+list(test[i]))) >30:
            col_more.append(i)
        else:
            col_less.append(i)
        #print i, len(np.unique(list(train[i])+list(test[i]))),len(np.setdiff1d(np.unique(test[i]),np.unique(train[i])))


In [66]:
test.type.value_counts()

E    2940
A    1171
B     351
C     289
F      26
D      21
H       3
Name: type, dtype: int64

In [67]:
train.type.value_counts()

E    5734
A    2316
B     638
C     562
F      61
D      47
H       7
G       1
Name: type, dtype: int64

In [68]:

train['creation_date']  = pd.to_datetime(train['creation_date'],format='%Y%m%d', errors='ignore')
train['start_date']  = pd.to_datetime(train['start_date'],format='%Y%m%d', errors='ignore')
train['sell_date']  = pd.to_datetime(train['sell_date'],format='%Y%m%d', errors='ignore')

test['creation_date']  = pd.to_datetime(test['creation_date'],format='%Y%m%d', errors='ignore')
test['start_date']  = pd.to_datetime(test['start_date'],format='%Y%m%d', errors='ignore')
test['sell_date']  = pd.to_datetime(test['sell_date'],format='%Y%m%d', errors='ignore')

In [69]:
train['sell_date_dow']  = train.sell_date.dt.dayofweek
test['sell_date_dow']  = test.sell_date.dt.dayofweek

train['sell_date_month']  = train.sell_date.dt.month
test['sell_date_month']  = test.sell_date.dt.month

train['sell_date_dayofyear']  = train.sell_date.dt.dayofyear
test['sell_date_dayofyear']  = test.sell_date.dt.dayofyear

train['start_date_dayofyear']  = train.start_date.dt.day
test['start_date_dayofyear']  = test.start_date.dt.day


train['sell_date_year']  = train.sell_date.dt.year
test['sell_date_year']  = test.sell_date.dt.year

train['sell_date_week_number']  = train.sell_date.dt.week
test['sell_date_week_number']  = test.sell_date.dt.week


In [70]:

train['difference_s_s'] = (train.start_date - train.sell_date).apply(lambda x: int(str(x).split(' ')[0]) )
test['difference_s_s'] = (test.start_date - test.sell_date).apply(lambda x: int(str(x).split(' ')[0]) )

test['difference_s_s'] = test['difference_s_s'].astype(int)
train['difference_s_s'] = train['difference_s_s'].astype(int)

train['difference_c_s'] = (train.creation_date - train.sell_date).apply(lambda x: int(str(x).split(' ')[0]) )
test['difference_c_s'] = (test.creation_date - test.sell_date).apply(lambda x: int(str(x).split(' ')[0]) )

test['difference_c_s'] = test['difference_c_s'].astype(int)
train['difference_c_s'] = train['difference_c_s'].astype(int)

In [71]:
train['return1'] = (train['sold'] -train['bought'])*1.0/train['bought']
test['return1'] = (test['sold'] -test['bought'])*1.0/test['bought']

total = pd.concat((train,test))

k = total[['desk_id','currency','portfolio_id']].groupby(['desk_id','currency']).agg('count').reset_index()
k = k.groupby('desk_id').agg('count')['currency'].reset_index()
k.columns = ['desk_id','desk_unique_count']
train = train.merge(k,on='desk_id',how='left')
test = test.merge(k,on='desk_id',how='left')


k1 = total[['sell_date_month','sell_date_year','desk_id','return1']].groupby(['desk_id','sell_date_month','sell_date_year']).agg('mean')['return1'].reset_index()
k1.columns = ['desk_id','sell_date_month','sell_date_year','mean_val_sell_date_month']
train = train.merge(k1,on=['desk_id','sell_date_month','sell_date_year'],how='left')
test = test.merge(k1,on=['desk_id','sell_date_month','sell_date_year'],how='left')

k1 = total[['sell_date_month','return1']].groupby(['sell_date_month']).agg('mean')['return1'].reset_index()
k1.columns = ['sell_date_month','mean_val_sell_date_month']
train = train.merge(k1,on='sell_date_month',how='left')

test = test.merge(k1,on='sell_date_month',how='left')
k1 = total[['start_date_dayofyear','return1']].groupby(['start_date_dayofyear']).agg('mean')['return1'].reset_index()
k1.columns = ['start_date_dayofyear','mean_val_start_date_dayofyear']
train = train.merge(k1,on='start_date_dayofyear',how='left')
test = test.merge(k1,on='start_date_dayofyear',how='left')


In [72]:
col_more = ['return1']
for enu,i in enumerate(col_more):
    print i
    
    total=pd.concat((train[[i,'sell_date_month','sell_date_year','currency']],test[[i,'sell_date_month','sell_date_year','currency']]))
    k = total.groupby(['sell_date_month','sell_date_year','currency']).agg('min').reset_index()
    k.columns = ['sell_date_month','sell_date_year','currency',i+'_min']
    train = train.merge(k,on=['sell_date_month','sell_date_year','currency'],how='left')
    test = test.merge(k,on=['sell_date_month','sell_date_year','currency'],how='left')
    
col_more = ['sold','bought']
col_more.append('sell_date_week_number')
train = train.drop(col_more,axis=1)
test = test.drop(col_more,axis=1)

y  = train['return'].values
train = train.drop('return',axis =1)

train['euribor_rate'] = np.log1p(train.euribor_rate)+1
test['euribor_rate'] = np.log1p(test.euribor_rate)+1


return1


In [73]:
train.replace(np.nan,-1,inplace=True)
test.replace(np.nan,-1,inplace=True)

p_id = test.portfolio_id.values

train = train.drop(['creation_date','start_date','sell_date','country_code'],axis =1)
test = test.drop(['creation_date','start_date','sell_date','country_code'],axis =1)

In [74]:
text_columns = []
for f in train.columns:
    if train[f].dtype== 'object':    
        text_columns.append(f)            
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))    


In [75]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.metrics import  roc_auc_score
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0,depth =8):
        params = {}
        params["objective"] = "reg:linear"
        params['eval_metric'] = 'rmse'
        params["eta"] = 0.01 #0.00334
        params["min_child_weight"] = 1
        params["subsample"] = 0.8
        params["colsample_bytree"] = 0.3
        params["silent"] = 1
        params["max_depth"] = depth
        params["seed"] = seed_val
        #params["max_delta_step"] = 2
        #params["gamma"] = 0.5
        num_rounds = 5000 #2500

        plst = list(params.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        if test_y is not None:
                xgtest = xgb.DMatrix(test_X, label=test_y)
                watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
                model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds= 500)
        else:
                xgtest = xgb.DMatrix(test_X)
                #xgtest1 = xgb.DMatrix(test_X1)
                model = xgb.train(plst, xgtrain, 4500)

        if feature_names:
                        create_feature_map(feature_names)
                        model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
                        importance = model.get_fscore(fmap='xgb.fmap')
                        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
                        imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
                        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
                        imp_df.to_csv("imp_feat.txt", index=False)

        pred_test_y = model.predict(xgtest)
        
        if test_y is not None:
                loss = r2_score(test_y, pred_test_y)
                print loss
        	return pred_test_y, loss
	else:
		return pred_test_y

In [76]:
"""
from sklearn.linear_model import LinearRegression
g_train = np.zeros(len(y))
g1_train = np.zeros(len(y))

g_test = np.zeros(test.shape[0])
g1_test = np.zeros(test.shape[0])


for i in np.unique(train.currency):
    print i
    ind_train = train.loc[train.currency==i].index
    ind_test = test.loc[test.currency==i].index
    train_train = train.iloc[ind_train]
    y_train1 = y[ind_train]
    test_test = test.iloc[ind_test]
    
    
    model = LinearRegression()
    model.fit(train_train,y_train1)
    pred_train = model.predict(train_train)
    pred_test = model.predict(test_test)
    g1_train[ind_train] = pred_train
    g1_test[ind_test] = pred_test
    
    
    k = np.log1p(y_train1+1)    
    model.fit(train_train,k)
    pred_train = np.expm1(model.predict(train_train))-1
    pred_test = np.expm1(model.predict(test_test))-1
    g_train[ind_train] = pred_train
    g_test[ind_test] = pred_test
    
train['g1'] = g1_train
test['g1'] = g1_test

train['g'] = g_train
test['g'] = g_test
"""

"\nfrom sklearn.linear_model import LinearRegression\ng_train = np.zeros(len(y))\ng1_train = np.zeros(len(y))\n\ng_test = np.zeros(test.shape[0])\ng1_test = np.zeros(test.shape[0])\n\n\nfor i in np.unique(train.currency):\n    print i\n    ind_train = train.loc[train.currency==i].index\n    ind_test = test.loc[test.currency==i].index\n    train_train = train.iloc[ind_train]\n    y_train1 = y[ind_train]\n    test_test = test.iloc[ind_test]\n    \n    \n    model = LinearRegression()\n    model.fit(train_train,y_train1)\n    pred_train = model.predict(train_train)\n    pred_test = model.predict(test_test)\n    g1_train[ind_train] = pred_train\n    g1_test[ind_test] = pred_test\n    \n    \n    k = np.log1p(y_train1+1)    \n    model.fit(train_train,k)\n    pred_train = np.expm1(model.predict(train_train))-1\n    pred_test = np.expm1(model.predict(test_test))-1\n    g_train[ind_train] = pred_train\n    g_test[ind_test] = pred_test\n    \ntrain['g1'] = g1_train\ntest['g1'] = g1_test\n\ntra

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)


In [78]:

from sklearn.metrics import r2_score
from sklearn.cross_validation import StratifiedKFold
np.random.seed(0) # seed to shuffle the train set
n_folds = 5
#0.903907431605
skf = list(StratifiedKFold(y, n_folds))
train2 = train.copy()
train2['pred'] = -1
train2['fold'] =-1
f = []
for i, (train1, test1) in enumerate(skf): 
        #print "Fold", i
        X_train = train.loc[train1]
        y_train = y[train1]
        X_test = train.loc[test1]
        y_test = y[test1]
        k = runXGB(X_train, y_train, X_test)
        train2.loc[test1,'pred']=k
        train2.loc[test1,'fold']=i
        print (r2_score(y_test,k))
        f.append(r2_score(y_test,k))
print np.mean(f)


0.582189107602
0.991106572534
0.996356250897
0.997846136988
0.993608128134
0.912221239231


In [79]:
"""
0.582189107602
0.991106572534
0.996356250897
0.997846136988
0.993608128134
0.912221239231

"""


'\n0.582189107602\n0.991106572534\n0.996356250897\n0.997846136988\n0.993608128134\n0.912221239231\n\n'

In [80]:
final_prediction = []
for i in range(0,100):
    if i%5==0:
        print i
    k = (y-np.min(y)) *1.0/(np.max(y)-np.min(y))
    pred6 = runXGB(train,k , test,depth =10,seed_val = i)
    pred6 = pred6*(np.max(y)-np.min(y)) + np.min(y)
    k = (y-np.min(y)) *1.0/(np.max(y)-np.min(y))
    pred7 = runXGB(train,k , test,depth =12,seed_val =i)
    pred7 = pred7*(np.max(y)-np.min(y)) + np.min(y)
    k = (y-np.min(y)) *1.0/(np.max(y)-np.min(y))
    pred8 = runXGB(train,k , test,depth =14,seed_val =i)
    pred8 = pred8*(np.max(y)-np.min(y)) + np.min(y)
    pred6_7 =(pred6+pred7+pred8)*1.0/3 
    final_prediction.append(pred6)
    final_prediction.append(pred7)
    final_prediction.append(pred8)


0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


In [89]:
pd.DataFrame({'portfolio_id':p_id,'return':np.median(final_prediction,axis = 0)}).to_csv('100_models.csv',index =False)

In [82]:
np.mean(final_prediction,axis = 0)

array([ 0.02519561,  0.0272034 ,  0.0245437 , ...,  0.0058769 ,
        0.00534486,  0.00586923], dtype=float32)

In [84]:
pred6

array([ 0.02514424,  0.02621793,  0.02434616, ...,  0.00591265,
        0.00535696,  0.00586584], dtype=float32)

In [85]:
pred7

array([ 0.02511298,  0.02619646,  0.02439666, ...,  0.0058863 ,
        0.00533841,  0.00588477], dtype=float32)

In [86]:
pred8

array([ 0.02509124,  0.02617314,  0.02439369, ...,  0.00587329,
        0.00535029,  0.00589438], dtype=float32)